

> Import libraries to use



In [77]:
import numpy as np

>  # Introduction to numpy (Skip if you already are familiar)

>> Creating a 1D array

In [78]:
a = np.array([1,2,3,4])
print(a)

[1 2 3 4]


>> Creating a 2D array


In [79]:
a = np.array([[1,2],[3,4]])
print(a)

[[1 2]
 [3 4]]


>> Creating an array full of zeros


In [80]:
a = np.zeros(shape=(10))
print(a)
a = np.zeros(shape=(5,2))
print(a)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]


>> Infinity in numpy

In [81]:
print(np.inf)

inf


>> Max and Argmax

In [82]:
a = np.array([2,1,4,3])
print(np.max(a))
print(np.argmax(a))

4
2


>> From list to Numpy

In [83]:
l = [1,2,3,4]
print(l)
print(np.asarray(l))

[1, 2, 3, 4]
[1 2 3 4]


>> Random in numpy

In [84]:
# Array of Random integers ranging from 1 to 10 (with any size you want)
a = np.random.randint(low=1, high=10, size=(5,2))
print(a)

# Array of random elements of a list with any size you want
a = np.random.choice([0,1,2], size=(2,))

[[8 1]
 [7 9]
 [4 3]
 [5 1]
 [7 9]]


>> Shapes in numpy

In [85]:
a = np.random.randint(low=1, high=5, size=(4,2))
print(a.shape)
print(a)

# Reshape a to a vector of shape = (8,1)
a = a.reshape((8,1))
print(a.shape)
print(a)

(4, 2)
[[2 3]
 [2 4]
 [2 3]
 [3 3]]
(8, 1)
[[2]
 [3]
 [2]
 [4]
 [2]
 [3]
 [3]
 [3]]


# Pre-defined utilities

In [87]:

int_to_char = {
    0 : 'u',
    1 : 'r',
    2 : 'd',
    3 : 'l'
}

policy_one_step_look_ahead = {
    0 : [-1,0],
    1 : [0,1],
    2 : [1,0],
    3 : [0,-1]
}

def policy_int_to_char(pi,n):

    pi_char = ['']

    for i in range(n):
        for j in range(n):

            if i == 0 and j == 0 or i == n-1 and j == n-1:

                continue

            pi_char.append(int_to_char[pi[i,j]])

    pi_char.append('')

    return np.asarray(pi_char).reshape(n,n)

# 1- Policy evaluation

In [89]:
import numpy as np

def policy_evaluation(n, pi, v, Gamma, threshhold):
    """
    This function returns the value function that follows the policy pi.
    :param n: Size of the gridworld (nxn)
    :param pi: The policy array of shape (n, n), with integer actions
    :param v: Initial value function of shape (n, n)
    :param Gamma: Discount factor (γ)
    :param threshhold: The stopping threshold for convergence
    :return: Updated value function v that follows policy pi
    """
    
    # Initialize a flag to check for convergence
    delta = threshhold + 1  # Setting delta larger than threshold initially

    while delta > threshhold:
        delta = 0
        
        # Iterate over all states in the grid
        for i in range(n):
            for j in range(n):
                
                # If we're at terminal states (like top-left or bottom-right), skip them
                if (i == 0 and j == 0) or (i == n - 1 and j == n - 1):
                    continue
                
                old_v = v[i, j]  # Store current value of state (i, j)

                # Get the action from the policy for the current state
                action = pi[i, j]
                
                # Get the movement corresponding to the action
                move = policy_one_step_look_ahead[action]
                new_i, new_j = i + move[0], j + move[1]
                
                # Boundaries check, ensuring we stay inside the grid
                if new_i < 0 or new_i >= n or new_j < 0 or new_j >= n:
                    new_i, new_j = i, j
                
                # Update the value of the state (i, j) based on Bellman's equation
                v[i, j] = -1 + Gamma * v[new_i, new_j]  # Reward of -1 for every move

                # Track the maximum difference for convergence checking
                delta = max(delta, abs(old_v - v[i, j]))

    return v


# 2- Policy improvement

In [63]:
import numpy as np

def policy_improvement(n, pi, v, Gamma):
    """
    This function returns the new policy by acting in a greedy manner.
    The function also returns a flag indicating if the new policy is the same as the input policy.
    
    :param n: Size of the gridworld (nxn)
    :param pi: Current policy array of shape (n, n), with integer actions
    :param v: Current value function of shape (n, n)
    :param Gamma: Discount factor (γ)
    :return: new_pi (improved policy), policy_stable (True if unchanged, False otherwise)
    """
    
    # Initialize a new policy and a flag for policy stability
    new_pi = np.copy(pi)
    policy_stable = True
    
    # Iterate over all states in the grid
    for i in range(n):
        for j in range(n):
            
            # If we're at terminal states (like top-left or bottom-right), skip them
            if (i == 0 and j == 0) or (i == n - 1 and j == n - 1):
                continue
            
            # Get the current action from the policy
            old_action = pi[i, j]
            
            # Evaluate all possible actions and find the best (greedy) action
            action_values = []
            for action in range(4):  # 4 possible actions: u, r, d, l
                
                move = policy_one_step_look_ahead[action]
                new_i, new_j = i + move[0], j + move[1]
                
                # Boundaries check, ensuring we stay inside the grid
                if new_i < 0 or new_i >= n or new_j < 0 or new_j >= n:
                    new_i, new_j = i, j
                
                # Compute the value of taking this action
                action_value = -1 + Gamma * v[new_i, new_j]  # Reward of -1 for every move
                action_values.append(action_value)
            
            # Select the action that maximizes the value
            best_action = np.argmax(action_values)
            
            # Update the policy with the best action
            new_pi[i, j] = best_action
            
            # Check if the policy has changed
            if best_action != old_action:
                policy_stable = False
    
    return new_pi, policy_stable


# 3- Policy Initialization

In [64]:
import numpy as np

def policy_initialization(n):
    """
    This function returns an initial random policy for all states.
    :param n: Size of the gridworld (nxn)
    :return: Randomly initialized policy array of shape (n, n) with values 0 (up), 1 (right), 2 (down), 3 (left)
    """
    
    # Initialize a random policy with integers between 0 and 3 representing actions
    pi = np.random.randint(0, 4, size=(n, n))
    
    # Set terminal states (0,0) and (n-1,n-1) to neutral actions (e.g., no movement)
    pi[0, 0] = -1  # Top-left terminal state
    pi[n-1, n-1] = -1  # Bottom-right terminal state
    
    return pi


# 4- Policy Iteration algorithm

In [65]:
def policy_iteration(n,Gamma,threshhold):

    pi = policy_initialization(n=n)

    v = np.zeros(shape=(n,n))

    while True:

        v = policy_evaluation(n=n,v=v,pi=pi,threshhold=threshhold,Gamma=Gamma)

        pi , pi_stable = policy_improvement(n=n,pi=pi,v=v,Gamma=Gamma)

        if pi_stable:

            break

    return pi , v

# Main Code to Test

In [66]:
n = 4

Gamma = [0.8,0.9,1]

threshhold = 1e-4

for _gamma in Gamma:

    pi , v = policy_iteration(n=n,Gamma=_gamma,threshhold=threshhold)

    pi_char = policy_int_to_char(n=n,pi=pi)

    print()
    print("Gamma = ",_gamma)

    print()

    print(pi_char)

    print()
    print()

    print(v)


TypeError: 'NoneType' object is not subscriptable